# start

In [1]:
# Import
from google.colab import files
from re import X
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler, Binarizer
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, balanced_accuracy_score, cohen_kappa_score,
                             hamming_loss, jaccard_score, roc_auc_score, log_loss, brier_score_loss, precision_recall_curve, roc_curve, auc)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline

In [2]:
# Upload spreadsheet
uploaded = files.upload()

Saving HB_preprocessed.xlsx to HB_preprocessed.xlsx


# Preprocessing

In [7]:
df = pd.read_excel("New_Data_HB (1).xlsx")

In [8]:
# Binary encoding
df["Adjuvant Systemic Therapy"] = df["Adjuvant Systemic Therapy"] == "Yes"
df["Cholecystitis"] = df["Cholecystitis"] == "Yes"
df["Cirrhosis"] = df["Cirrhosis"] == "Yes"
df["Diabetes"] = df["Diabetes"] == "Yes"
df["Lymphovascular Invasion"] = df["Lymphovascular Invasion"] == "Yes"
df["Major Liver Resection"] = df["Major Liver Resection"] == "Yes"
df["Mutlifocal Disease"] = df["Mutlifocal Disease"] == "Yes"
df["Neoadjuvant Systemic Therapy"] = df["Neoadjuvant Systemic Therapy"] == "Yes"
df["Perineural Invasion"] = df["Perineural Invasion"] == "Yes"
df["Sex"] = df["Sex"] == "Male"
df["Steatosis"] = df["Steatosis"] == "Yes"
df["Viral Hepatitis"] = df["Viral Hepatitis"] == "Yes"
df["Palliative Systemic Therapy"] = df["Palliative Systemic Therapy"] == "Yes"
df['BAP1'] = df["BAP1"] == "Mutant"
df['IDH1'] = df["IDH1"] == "Mutant"
df['KRAS'] = df["KRAS"] == "Mutant"
df['TP53'] = df["TP53"] == "Mutant"
df['SMAD4'] = df["SMAD4"] == "Mutant"

In [10]:
# One hot encoding for categorical columns. some contain N/A as a category
categorical_cols = ["Tumor Stage", "Sample Type", "Subgroup", "Resection Margin", "Race", "Primary Tumor Site",  "Node Status ", "Suspicious Node (imaging)"]

df = pd.get_dummies(df, columns=categorical_cols, drop_first=True, dummy_na=False)


In [15]:
# Scale continuous columns
continuous_cols = ["BMI", "Fraction Genome Altered", "TMB (nonsynonymous)", "Tumor Purity", "Age at Diagnosis"]

scaler = StandardScaler()

# Function to scale only the specified columns
def scale_selected_columns(df, scaler):
    df_scaled = df.copy()
    df_scaled[continuous_cols] = scaler.fit_transform(df_scaled[continuous_cols])
    return df_scaled

df = scale_selected_columns(df, StandardScaler())

In [19]:
df = df.dropna()

In [ ]:
df.head()

In [20]:
# Download preprocessed file
df.to_excel("HB_preprocessed.xlsx", index=False)
files.download("HB_preprocessed.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Descriptives

In [ ]:
df = pd.read_excel("New_Data_HB (1).xlsx")

In [ ]:
# Continuous cols
columns = ["MSI Score", "Sample ID", "BMI", "Tumor Purity",
           "Age at Diagnosis", "Fraction Genome Altered", "TMB (nonsynonymous)"]

summary_stats = df[columns].describe().T
summary_stats = summary_stats.rename(columns={"25%": "Q1", "50%": "Q2 (Median)", "75%": "Q3"})
summary_stats = summary_stats[["mean", "std", "min", "Q1", "Q2 (Median)", "Q3", "max"]]
summary_stats.to_excel("HB_descriptives_cont.xlsx")
files.download("HB_descriptives_cont.xlsx")

In [ ]:
# Ordinal
def calculate_value_counts_and_percentage(df):
    result = []
    for col in df.columns:
        value_counts = df[col].value_counts()
        percentage = (df[col].value_counts(normalize=True) * 100).round(2)

        temp_df = pd.DataFrame({
            'Feature': col,
            'Value': value_counts.index,
            'Count': value_counts.values,
            'Percentage (%)': percentage.values
        })
        result.append(temp_df)

    # Concatenate all DataFrames in the list
    final_df = pd.concat(result, ignore_index=True)

    # Save the final DataFrame to a CSV file
    #final_df.to_csv(output_filename, index=False)
    #print(final_df)

    return final_df
df = df.drop(columns=["MSI Score", "Sample ID", "BMI", "Tumor Purity", "Age at Diagnosis", "Fraction Genome Altered", "TMB (nonsynonymous)"])
result = calculate_value_counts_and_percentage(df)

In [ ]:
result.to_excel("HB_descriptives_ordinal.xlsx")
files.download("HB_descriptives_ordinal.xlsx")

# classification models

## Function definitions



---



In [3]:
# Function to find best Hyper Parameters using GridSearchCV
def train_model(model, ds):
  if ds == "LVI":
    X = X_train_LVI
    y = y_train_LVI
  elif ds == "PNI":
    X = X_train_PNI
    y = y_train_PNI
  else:
    raise ValueError("Invalid data type. Use 'LVI' or 'PNI'.")

  dt = GridSearchCV(models[model], param_grid = param_grids[model], scoring='roc_auc', cv=5)
  dt.fit(X, y)

  output = f"""
  MODEL : {model} {ds}
  best cv score: {dt.best_score_}
  best cv params: {dt.best_params_}
  """
  print(output)
  return dt


# Function to evaluate models
def analyze_cls_model(model, model_name, metrics_df, data, print_output=False):
    if data == "LVI":
        y_test = y_test_LVI
        X_test = X_test_LVI
    elif data == "PNI":
        y_test = y_test_PNI
        X_test = X_test_PNI
    else:
      raise ValueError("Invalid data type. Use 'LVI' or 'PNI'.")


    pred = model.predict(X_test)
        # Assume `pred` are your predictions and `y_test_cls` are the true class labels
    conf_matrix = confusion_matrix(y_test, pred)
    TP = conf_matrix[1, 1]  # True Positive
    TN = conf_matrix[0, 0]  # True Negative
    FP = conf_matrix[0, 1]  # False Positive
    FN = conf_matrix[1, 0]  # False Negative

    # Sensitivity, hit rate, recall, or true positive rate
    sensitivity = TP / (TP + FN)
    # Specificity or true negative rate
    specificity = TN / (TN + FP)
    # Precision or positive predictive value
    ppv = TP / (TP + FP)
    # Negative predictive value
    npv = TN / (TN + FN)
    # Fall out or false positive rate
    fpr = FP / (FP + TN)
    # False negative rate
    fnr = FN / (TP + FN)
    # True positive rate
    tpr = TP / (TP + FN)
    # False discovery rate
    fdr = FP / (TP + FP)

    metrics = {
        'Model': model_name,
        'Sensitivity': sensitivity,
        'Accuracy': accuracy_score(y_test, pred),
        'Precision': precision_score(y_test, pred, average='binary'),
        "Specificity": specificity,
        'Recall': recall_score(y_test, pred, average='binary'),
        'F1 Score': f1_score(y_test, pred, average='binary'),
        'TP': TP,
        'TN': TN,
        'FP': FP,
        'FN': FN,
        'PPV': ppv,
        'NPV': npv,
        'FPR': fpr,
        'FNR': fnr,
        'TPR': tpr,
        'FDR': fdr,
        'Balanced Accuracy': balanced_accuracy_score(y_test, pred),
        'Cohen Kappa Score': cohen_kappa_score(y_test, pred),
        'Hamming Loss': hamming_loss(y_test, pred),
        'Jaccard Score': jaccard_score(y_test, pred, average='binary')
    }

    # Probability-based metrics if applicable
    if hasattr(model, 'predict_proba'):
        prob_pred = model.predict_proba(X_test)[:, 1]
        print("1.pred: ", prob_pred)
        metrics.update({
            'ROC AUC': roc_auc_score(y_test, prob_pred)
        })
    else :
      # If not probability-based, use decision function
      prob_pred = model.decision_function(X_test)
      print("2.pred: ", prob_pred)
      metrics.update({
          'ROC AUC': roc_auc_score(y_test, prob_pred)
      })
      print("pred: ", prob_pred)

    def model_predict_proba(X):
        return model.predict_proba(X)

    if print_output:
        output = f"""Specificity (TNR): {specificity:.4f}
PPV (Positive Predictive Value): {ppv:.4f}
NPV (Negative Predictive Value): {npv:.4f}
FPR (False Positive Rate): {fpr:.4f}
FNR (False Negative Rate): {fnr:.4f}
TPR (True Positive Rate): {tpr:.4f}
FDR (False Discovery Rate): {fdr:.4f}"""
        print(f"\nModel: {model_name}")
        print(f"Confusion Matrix:\n{conf_matrix}")
        print(f"Accuracy: {metrics['Accuracy']:.4f}")
        print(f"Precision: {metrics['Precision']:.4f}")
        print(f"Recall: {metrics['Recall']:.4f}")
        print(f"F1 Score: {metrics['F1 Score']:.4f}")
        print(f"Balanced Accuracy: {metrics['Balanced Accuracy']:.4f}")
        print(f"Cohen Kappa Score: {metrics['Cohen Kappa Score']:.4f}")
        print(f"Hamming Loss: {metrics['Hamming Loss']:.4f}")
        print(f"Jaccard Score: {metrics['Jaccard Score']:.4f}")
        print(output)
        if 'ROC AUC' in metrics:
            print(f"ROC AUC: {metrics['ROC AUC']:.4f}")
            print(f"Log Loss: {metrics['Log Loss']:.4f}")
            print(f"Brier Score Loss: {metrics['Brier Score Loss']:.4f}")
    print(f"\nModel: {model_name}")
    print(f"Confusion Matrix:\n{conf_matrix}")
    metrics_df = pd.concat([metrics_df, pd.DataFrame([metrics])], ignore_index=True)
    return metrics_df


# Generate the precision recall curves of a particular dataset.
# Used to determine best decision threholds
def plot_precision_recall_curve(ds):
  if ds == "LVI":
    X_train = X_train_LVI
    y_train = y_train_LVI
    X_test = X_test_LVI
    y_test = y_test_LVI
    models = best_LVI_models
  elif ds == "PNI":
    X_train = X_train_PNI
    y_train = y_train_PNI
    X_test = X_test_PNI
    y_test = y_test_PNI
    models = best_PNI_models
  else:
    raise ValueError("Invalid dataset. Use 'LVI' or 'PNI'.")


  for model_name, model in models.items():
      if hasattr(model, 'predict_proba'):
          prob_pred = model.predict_proba(X_train)[:, 1]
          precision, recall, thresholds = precision_recall_curve(y_train, prob_pred)
      else:
          # Handle models without predict_proba (e.g., SVM without probability=True)
          prob_pred = model.decision_function(X_train)
          precision, recall, thresholds = precision_recall_curve(y_train, prob_pred)

      plt.figure()
      plt.plot(recall, precision, marker='.')
      plt.xlabel('Recall')
      plt.ylabel('Precision')
      plt.title(f'Precision-Recall Curve for {model_name}-LVI')
      plt.grid(True)
      plt.show()


# Print the precision, recall and F1 scores of a model in a given range of recall values.
# Use this to find a good decision threshold after determining a rough range using the precision recall curves
def print_threshold_metrics_ranged(model_name, ds, recall_min,  recall_max):
  if ds == "LVI":
      X_train = X_train_LVI
      y_train = y_train_LVI
      model = best_LVI_models[model_name]
  elif ds == "PNI":
      X_train = X_train_PNI
      y_train = y_train_PNI
      model = best_PNI_models[model_name]
  else:
      raise ValueError("Invalid dataset. Use 'LVI' or 'PNI'.")
  if hasattr(model, 'predict_proba'):
          prob_pred = model.predict_proba(X_train)[:, 1]
          precision, recall, thresholds = precision_recall_curve(y_train, prob_pred)
  else:
      # Handle models without predict_proba (e.g., SVM without probability=True)
      prob_pred = model.decision_function(X_train)
      precision, recall, thresholds = precision_recall_curve(y_train, prob_pred)

  f1_scores = 2 * (precision * recall) / (precision + recall)

  # Find indices where recall is within the specified range
  indices = np.where((recall >= recall_min) & (recall <= recall_max))[0]

  print("Recall\tPrecision\tF1 Score\tThreshold")
  for i in indices:
      print(f"{recall[i]:.2f}\t\t{precision[i]:.2f}\t\t{f1_scores[i]:.2f}\t\t{thresholds[i]:.4f}")


# Print recall, precision and F1 score of a particular threshold
def print_threshold_metrics(model_name, ds, threshold):
  if ds == "LVI":
      X_train = X_train_LVI
      y_train = y_train_LVI
      model = best_LVI_models[model_name]
  elif ds == "PNI":
      X_train = X_train_PNI
      y_train = y_train_PNI
      model = best_PNI_models[model_name]
  else:
      raise ValueError("Invalid dataset. Use 'LVI' or 'PNI'.")


  if hasattr(model, 'predict_proba'):
          prob_pred = model.predict_proba(X_train_PNI)[:, 1]
          precision, recall, thresholds = precision_recall_curve(y_train_PNI, prob_pred)
  else:
      # Handle models without predict_proba (e.g., SVM without probability=True)
      prob_pred = model.decision_function(X_train_PNI)
      precision, recall, thresholds = precision_recall_curve(y_train_PNI, prob_pred)

  # Find the closest threshold index
  closest_index = np.argmin(np.abs(thresholds - threshold))

  print(f"Model: {model_name} {ds}")
  print(f"Threshold: {thresholds[closest_index]:.4f}")
  print(f"Precision: {precision[closest_index]:.2f}")
  print(f"Recall: {recall[closest_index]:.2f}")
  print(f"F1 Score: {2 * (precision[closest_index] * recall[closest_index]) / (precision[closest_index] + recall[closest_index]):.2f}")
  print("-" * 20)


# Function to adjust the thresholds
def adjust_threshold(model, ds, threshold):
  binarizer = Binarizer(threshold=best_threshold)
  if ds == "LVI":
      best_LVI_models[model] = Pipeline([('binarizer', binarizer), ('model', best_LVI_models[model])])
  elif ds == "PNI":
      best_PNI_models[model] = Pipeline([('binarizer', binarizer), ('model', best_PNI_models[model])])
  else:
    raise ValueError("Invalid dataset. Use 'LVI' or 'PNI'.")

# Function to calculate AUC scores for models
def calculate_auc_scores(models, X_test, y_test):
    auc_scores = {}
    for model_name, model in models.items():
        if hasattr(model, 'predict_proba'):
          prob_pred = model.predict_proba(X_test)[:, 1]
        else:
          prob_pred = model.decision_function(X_test)
        roc_auc = roc_auc_score(y_test, prob_pred)
        auc_scores[model_name] = roc_auc
    return auc_scores


# Function to plot ROC curves for multiple models
def plot_roc_curves(models, X_test, y_test, title):
    plt.figure(figsize=(10, 6))
    plt.title(title)
    for model_name, model in models.items():
        if hasattr(model, 'predict_proba'):
          prob_pred = model.predict_proba(X_test)[:, 1]
        else:
          prob_pred = model.decision_function(X_test)
        fpr, tpr, _ = roc_curve(y_test, prob_pred)
        roc_auc = roc_auc_score(y_test, prob_pred)
        plt.plot(fpr, tpr, label=f"{model_name} (AUC = {roc_auc:.4f})")
    plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='best')
    plt.show()


# Function to plot ROC curves of models of a particular ds, each plot contains 3 models. Plots sorted in descending order.
def plot_roc_groupsofthree(ds):
  if ds == "LVI":
    models = best_LVI_models
    X_test = X_test_LVI
    y_test = y_test_LVI
  elif ds == "PNI":
    models = best_PNI_models
    X_test = X_test_PNI
    y_test = y_test_PNI
  else:
    raise ValueError("Invalid dataset. Use 'LVI' or 'PNI'.")
  # Calculate AUC scores
  auc_scores = calculate_auc_scores(models, X_test, y_test)
  # Sort models by AUC scores in descending order
  sorted_models = dict(sorted(models.items(), key=lambda item: auc_scores[item[0]], reverse=True))
  # Split the sorted models into groups of 3
  sorted_models_groups = [dict(list(sorted_models.items())[i:i+3]) for i in range(0, len(sorted_models), 3)]
  # Plot ROC curves for each group
  for i, group in enumerate(sorted_models_groups):
      plot_roc_curves(group, X_test, y_test, f"{ds} - ROC Curves (Group {i+1})")

In [ ]:
# Define the classifiers
models = {
    'KNN': KNeighborsClassifier(),
    'MLP': MLPClassifier(max_iter = 5000),
    'SVM': SVC(probability=True),
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(random_state= 125),
    'Random Forest': RandomForestClassifier(random_state = 125),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Gaussian Naive Bayes': GaussianNB()
}

param_grids = {
    'KNN': {
        'n_neighbors': [90, 95, 97, 99, 101],  # Centered around best values
        'leaf_size': [10, 15, 20, 25, 30],  # Focus on lower values
        'p': [1, 2],  # Best params used Manhattan (p=1), but keep 2
        'weights': ['uniform', 'distance'],  # Best params used both
        'metric': ['manhattan', 'minkowski']  # Euclidean didn't perform well
    },
    'MLP': {
        'activation': ['tanh', 'relu'],  # Best params used tanh
        'hidden_layer_sizes': [(10,), (50,), (50, 50), (100,)],  # Best params used (10,) and (50,50)
        'learning_rate_init': [0.0001, 0.001, 0.01],  # Added 0.0001 for more fine-tuning
        'alpha': [0.0001, 0.001, 0.05],  # Best params used 0.0001 and 0.05
        'solver': ['sgd', 'adam'],  # Best params used sgd, but keep adam
        'learning_rate': ['adaptive', 'constant']  # Best params used adaptive
    },
    'SVM': {
        'C': [0.05, 0.1, 0.3, 1, 3],  # Best params used 0.1 and 0.3
        'kernel': ['linear', 'rbf'],  # Best params used linear, remove poly
        'gamma': ['scale', 'auto']  # No extreme gamma values needed
    },
    'Logistic Regression': {
        'C': [0.01, 0.1, 1],  # Best params used 0.01 and 1
        'solver': ['liblinear', 'saga'],  # Best params used these
        'penalty': ['l1', 'l2']  # Both were used in best params
    },
    'Decision Tree': {
        'max_depth': [3, 5, 10, 15, None],  # Best params used 3, remove extreme values
        'min_samples_split': [2, 5, 10, 20],  # Slight refinement
        'min_samples_leaf': [2, 4, 8, 10]  # Best params used 8 and 10
    },
    'Random Forest': {
        'n_estimators': [50, 100, 150, 200],  # Best params used 150 and 200
        'max_depth': [1, 3, 10, None],  # Best params used 1 and 10
        'min_samples_split': [2, 10, 20, 40],  # Best params used 2 and 40
        'bootstrap': [True, False]  # Added for additional tuning
    },
    'Gradient Boosting': {
        'n_estimators': [20, 50, 100],  # Best params used 50
        'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Best params used 0.05 and 0.2
        'max_depth': [1, 2, 3]  # Best params used 1 and 2, remove extreme values
    },
    'AdaBoost': {
        'n_estimators': [10, 50, 100],  # Best params used 50 and 100
        'learning_rate': [0.1, 0.5, 0.75, 1.0],  # Best params used 1.0 and 0.75
        'algorithm': ['SAMME', 'SAMME.R']  # Added for optimization
    },
    'Gaussian Naive Bayes': {
        'var_smoothing': [1e-11, 1e-9, 1e-7, 1e-5]  # Removed extreme values like 1e-50
    }
}

## Code



---



In [4]:
#copy sheet to df
df = pd.read_excel("HB_preprocessed.xlsx")
LVI_df = df.drop(columns = "Perineural Invasion")
PNI_df = df.drop(columns = "Lymphovascular Invasion")

In [5]:
LVI_df = LVI_df.drop("Sample ID", axis = 1)
PNI_df = PNI_df.drop("Sample ID", axis = 1)

In [6]:
# train test split
X_train_LVI, X_test_LVI, y_train_LVI, y_test_LVI = train_test_split(
    LVI_df.drop(columns=["Lymphovascular Invasion"]),
    LVI_df["Lymphovascular Invasion"],
    test_size=0.3,
    random_state=123
)

X_train_PNI, X_test_PNI, y_train_PNI, y_test_PNI = train_test_split(
    PNI_df.drop(columns=["Perineural Invasion"]),
    PNI_df["Perineural Invasion"],
    test_size=0.3,
    random_state=42
)


In [ ]:
# Find best hyper parameters and print them to terminal
warnings.filterwarnings("ignore", category=UserWarning)
for model_name, _ in models.items():
    train_model(model_name, "LVI")
    train_model(model_name, "PNI")

In [138]:
# Define the classification models using the best parameters obtained using GridSearchCV
best_LVI_models = {
    'KNN': KNeighborsClassifier(leaf_size=10, metric='manhattan', n_neighbors=90, p=1, weights='uniform'),
    'MLP': MLPClassifier(activation='tanh', alpha=0.0001, hidden_layer_sizes=(10,), learning_rate='constant', learning_rate_init=0.0001, solver='adam', max_iter=5000, random_state=11),
    #'SVM': SVC(probability=True, gamma='scale', kernel='linear', C=0.3, random_state=11),
    'Logistic Regression': LogisticRegression(C=0.01, penalty='l2', solver='saga'),
    'Decision Tree': DecisionTreeClassifier(random_state=125, max_depth=3, min_samples_leaf=10, min_samples_split=2),
    'Random Forest': RandomForestClassifier(random_state=125, bootstrap=True, max_depth=1, min_samples_split=2, n_estimators=200),
    'Gradient Boosting': GradientBoostingClassifier(learning_rate=0.2, max_depth=1, n_estimators=50),
    'AdaBoost': AdaBoostClassifier(algorithm='SAMME', learning_rate=1.0, n_estimators=50),
    'Gaussian Naive Bayes': GaussianNB(var_smoothing=1e-11)
}

In [142]:
best_PNI_models = {
    'KNN': KNeighborsClassifier(leaf_size=10, metric='manhattan', n_neighbors=97, p=1, weights='distance'),
    #'MLP': MLPClassifier(activation='relu', alpha=0.0001, hidden_layer_sizes=(50, 50), learning_rate='constant', learning_rate_init=0.01, solver='adam', max_iter=5000, random_state=11),
    #'SVM': SVC(probability=True, gamma='scale', kernel='rbf', C=3, random_state=11),
    'Logistic Regression': LogisticRegression(C=1, penalty='l2', solver='liblinear'),
    'Decision Tree': DecisionTreeClassifier(random_state=125, max_depth=3, min_samples_leaf=8, min_samples_split=20),
    'Random Forest': RandomForestClassifier(random_state=125, bootstrap=True, max_depth=10, min_samples_split=40, n_estimators=150),
    'Gradient Boosting': GradientBoostingClassifier(learning_rate=0.05, max_depth=2, n_estimators=50),
    'AdaBoost': AdaBoostClassifier(algorithm='SAMME', learning_rate=1.0, n_estimators=100),
    'Gaussian Naive Bayes': GaussianNB(var_smoothing=1e-11)
}

In [ ]:
# Fit models
for model_name, model in best_LVI_models.items():
    model.fit(X_train_LVI, y_train_LVI)
for model_name, model in best_PNI_models.items():
    model.fit(X_train_PNI, y_train_PNI)

In [ ]:
ds = "PNI"
plot_precision_recall_curve(ds)

In [ ]:
model_name = "Gradient Boosting"
ds = "PNI"
recall_min = 0.7  # Lower bound of recall range
recall_max = 0.97 # Upper bound of recall range
print_threshold_metrics_ranged(model_name, ds, recall_min, recall_max)

In [ ]:
model_name = "Gradient Boosting"
ds = "LVI"
threshold = 0.5662

print_threshold_metrics(model_name, ds, threshold)

In [92]:
# Adjust thresholds
# repeat for every threshold you want to change
best_threshold = 0.5662
model = "Gradient Boosting"
ds = "LVI"

adjust_threshold(model, ds, best_threshold)

In [ ]:
# Evaluate models and store metrics in a df
metrics_columns = [
    'Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC', 'Balanced Accuracy', 'Cohen Kappa Score',
    'Hamming Loss', 'Jaccard Score', 'Log Loss', 'Brier Score Loss', 'Specificity', 'Sensitivity',
    'PPV', 'NPV', 'FPR', 'FNR', 'TPR', 'FDR','TP','TN', 'FP','FN'
]
# Initialize the DataFrame
metrics_LVI = pd.DataFrame(columns=metrics_columns)
metrics_PNI = pd.DataFrame(columns=metrics_columns)

#Train and evaluate each model
for model_name, model in best_LVI_models.items():
    model.fit(X_train_LVI, y_train_LVI)
    metrics_LVI = analyze_cls_model(model, model_name, metrics_LVI, "LVI")
for model_name, model in best_PNI_models.items():
    model.fit(X_train_PNI, y_train_PNI)
    metrics_PNI = analyze_cls_model(model, model_name, metrics_PNI, "PNI")

In [ ]:
metrics_LVI

In [ ]:
metrics_PNI

In [96]:
# Download metrics
metrics_LVI.to_excel('metrics_LVI.xlsx', index=False)
files.download('metrics_LVI.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [97]:
metrics_PNI.to_excel('metrics_PNI.xlsx', index=False)
files.download('metrics_PNI.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [161]:
# Architecture of SKlearn MLP
mlp = best_LVI_models["MLP"]
mlp.fit(X_train_PNI, y_train_PNI)

hidden_layer_sizes = mlp.hidden_layer_sizes
number_of_hidden_layers = len(hidden_layer_sizes)
number_of_input_features = X_train_PNI.shape[1]

# Print the architecture
print("Input layer size:", number_of_input_features)
print("Hidden layer sizes:", hidden_layer_sizes)
print("Number of hidden layers:", number_of_hidden_layers)

Input layer size: 37
Hidden layer sizes: (10,)
Number of hidden layers: 1


In [ ]:
ds = "PNI"
plot_roc_groupsofthree(ds)

## Shap

### Function definitions

In [ ]:
!pip install shap

In [105]:
import shap

In [139]:
# Shap summary plot
def plot_shap(ds, model_name):
  # Name mapping for proper name display on plot
  #TODO: name mapping new columns
  name_mapping = {
      'Adjuvant Systemic Therapy': 'Adjuvant Systemic Therapy',
      'BMI': 'BMI',
      'Cholecystitis': 'Cholecystitis',
      'Cirrhosis': 'Cirrhosis',
      'Diabetes': 'Diabetes',
      'Age': 'Age',
      'Fraction Genome Altered': 'Fraction Genome Altered',
      'Major Liver Resection': 'Major Liver Resection',
      'MSI Score': 'MSI Score',
      'Multifocal Disease': 'Multifocal Disease',
      'Neoadjuvant Systemic Therapy': 'Neoadjuvant Systemic Therapy',
      'Palliative Systemic Therapy': 'Palliative Systemic Therapy',
      'Sex': 'Sex',
      'Steatosis': 'Steatosis',
      'TMB (nonsynonymous)': 'TMB (nonsynonymous)',
      'Tumor Purity': 'Tumor Purity',
      'Viral Hepatitis': 'Viral Hepatitis',
      'Resection Margin_R0': 'Resection Margin: R0',
      'Resection Margin_R1': 'Resection Margin: R1',
      'Resection Margin_R2': 'Resection Margin: R2',
      'Sample Type_Metastasis': 'Sample Type: Metastasis',
      'Sample Type_Primary': 'Sample Type: Primary',
      'Subgroup_Biphenotypic tumors': 'Subgroup: Biphenotypic tumors',
      'Subgroup_EHC': 'Subgroup: EHC',
      'Subgroup_GB': 'Subgroup: GB',
      'Subgroup_HCC': 'Subgroup: HCC',
      'Subgroup_IHC': 'Subgroup: IHC',
      'Tumor Stage_Moderately differentiated': 'Tumor Stage: Moderately differentiated',
      'Tumor Stage_Poorly differentiated': 'Tumor Stage: Poorly differentiated',
      'Tumor Stage_Well differentiated': 'Tumor Stage: Well differentiated',
      'Primary Tumor Site_Bile Duct': 'Primary Tumor Site: Bile Duct',
      'Primary Tumor Site_Gallbladder': 'Primary Tumor Site: Gallbladder',
      'Primary Tumor Site_Liver': 'Primary Tumor Site: Liver',
      'Race_Asian': 'Race: Asian',
      'Race_Black': 'Race: Black',
      'Race_Other': 'Race: Other',
      'Race_White': 'Race: White',
      # New columns
      'Age at Diagnosis': 'Age at Diagnosis',
      'Mutlifocal Disease': 'Multifocal Disease',  # corrected typo (Mutlifocal -> Multifocal)
      'IDH1': 'IDH1',
      'KRAS': 'KRAS',
      'TP53': 'TP53',
      'BAP1': 'BAP1',
      'SMAD4': 'SMAD4',
      'Lymphovascular Invasion': 'Lymphovascular Invasion',
      'Tumor Stage_Poorly differentiated': 'Tumor Stage: Poorly differentiated',
      'Tumor Stage_Well differentiated': 'Tumor Stage: Well differentiated',
      'Node Status _Negative': 'Node Status: Negative',
      'Node Status _Positive': 'Node Status: Positive',
      'Suspicious Node (imaging)_No': 'Suspicious Node (imaging): No',
      'Suspicious Node (imaging)_Yes': 'Suspicious Node (imaging): Yes',
      'Race_BLACK': 'Race: Black',
      'Race_OTHER': 'Race: Other',
      'Race_WHITE': 'Race: White',
      'Primary Tumor Site_Bile duct': 'Primary Tumor Site: Bile Duct'  # corrected case (Bile duct -> Bile Duct)
  }


  if ds == "LVI":
    X_train = X_train_LVI
    X_test = X_test_LVI
    model = best_LVI_models[model_name]
  elif ds == "PNI":
    X_train = X_train_PNI
    X_test = X_test_PNI
    model = best_PNI_models[model_name]
  else:
    raise ValueError("Invalid dataset. Use 'LVI' or 'PNI'.")

  X_train = pd.DataFrame(X_train)

  # Get feature names and map them
  feature_names = X_train.columns
  mapped_names = [name_mapping.get(name, name) for name in feature_names]

  background_data = shap.sample(X_train, 100)

  # Wrap the model's predict_proba method to handle feature names (doens't work for models without predict proba (SVM). uncomment the line)
  def model_wrapper(X):
      return model.predict_proba(pd.DataFrame(X, columns=feature_names))

  explainer = shap.KernelExplainer(model_wrapper, background_data)
  shap_values = explainer(X_test)

  # Select shap values for the first class (index 0) for all instances
  shap_values_class0 = shap_values[:, :, 0]
  result = (shap_values_class0, X_test, mapped_names)
  # Plot the summary plot for the first class
  print(f"Summary Plot for {model_name} Model {ds}")
  shap.summary_plot(shap_values_class0, X_test, feature_names=mapped_names, max_display=25)
  shap.summary_plot(shap_values_class0, X_test, feature_names=mapped_names, max_display=25, plot_type="bar")
  shap_values_expl = shap.Explanation(shap_values_class0[0], feature_names=mapped_names)
  shap.plots.bar(shap_values_expl, max_display=25)
  return result

### Code

In [ ]:
print("-" * 100)
print("PNI")
print("-" * 100)
shap_vals_PNI = {}
shap_vals_LVI = {}
for name, _ in best_PNI_models.items():
  shap_vals_PNI[name] = plot_shap("PNI", name)
print("-" * 100)
print("LVI")
print("-" * 100)
for name , _ in best_LVI_models.items():
  shap_vals_LVI[name] = plot_shap("LVI", name)

# TF MLP

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers, Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2

In [8]:
# Function to evaluate the TF mlp model
def evaluate_model(model, X_test, y_test):
    y_pred_probs = model.predict(X_test).ravel()
    y_pred = (y_pred_probs >= 0.5).astype(int)

    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)  # Sensitivity
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn + fp)

    # Compute ROC curve and AUC
    fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
    roc_auc = auc(fpr, tpr)

    # Print metrics
    print(f"F1 Score: {f1:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall (Sensitivity): {recall:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"AUC-ROC: {roc_auc:.4f}")
    print(f"Confusion Matrix: TN={tn}, FP={fp}, FN={fn}, TP={tp}")

    # Plot ROC curve
    plt.figure(figsize=(6, 5))
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc='lower right')
    plt.show()

    # Plot confusion matrix
    plt.figure(figsize=(5, 4))
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()


In [ ]:
# PNI_MLP
PNI_MLP = keras.Sequential([
    layers.Dense(78, input_shape=(37,), activation='relu', name='dense_17'),
    layers.Dense(39, activation='relu', name='dense_18'),
    layers.Dense(20, activation='relu', name='dense_19'),
    layers.Dense(10, activation='relu', name='dense_20'),
    layers.Dense(1, activation='sigmoid', name='dense_21')
])

PNI_MLP.layers[0] = layers.Dense(78, input_shape=(78,), activation='relu', name='dense_17', kernel_initializer='glorot_uniform', bias_initializer='zeros')
PNI_MLP.compile(optimizer=keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])

PNI_MLP.summary()
PNI_MLP.fit(X_train_PNI, y_train_PNI)
evaluate_model(PNI_MLP, X_test_PNI, y_test_PNI)

In [ ]:
# Define the model
LVI_MLP = Sequential([
    layers.Dropout(0.01, input_shape=(37,), name='dropout_14'),  # Increased dropout for regularization
    layers.Dense(64, activation='relu', kernel_regularizer=l2(0.01), name='dense_116'),  # Reduced units and added L2 regularization
    layers.BatchNormalization(name='batch_norm_1'),  # Added batch normalization
    layers.Dropout(0.3, name='dropout_15'),  # Added another dropout layer
    layers.Dense(32, activation='relu', kernel_regularizer=l2(0.01), name='dense_117'),  # Further reduced units
    layers.BatchNormalization(name='batch_norm_2'),  # Added batch normalization
    layers.Dropout(0.3, name='dropout_16'),  # Added another dropout layer
    layers.Dense(16, activation='relu', kernel_regularizer=l2(0.01), name='dense_118'),  # Further reduced units
    layers.BatchNormalization(name='batch_norm_3'),  # Added batch normalization
    layers.Dropout(0.3, name='dropout_17'),  # Added another dropout layer
    layers.Dense(1, activation='sigmoid', name='dense_120')  # Output layer
])

# Compile the model
LVI_MLP.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
LVI_MLP.summary()
LVI_MLP.fit(X_train_LVI, y_train_LVI)
evaluate_model(LVI_MLP, X_test_LVI, y_test_LVI)